# Deployment Benefits

### Introduction

Now that we have learned a little about deployments, it's time to take some time to further understand how deployments and Kubernetes in general can help us manage our application.  As we mentioned, Kubernetes helps us with deploying and managing our containers.  In this lesson, we'll see some of those benefits in action.

### Creating new containers

Let's take another look at our deployment file for creating a new API.

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: api-deployment
spec:
  replicas: 2
  selector:
    matchLabels:
      component: api
  template:
    metadata:
      labels:
        component: api
    spec:
      containers:
        - name: flask-api
          image: jek2141/flask_api
          ports:
          - containerPort: 5000
```

The key part to notice in our deployment above is that we just specified the number of replicas of our pods that we wanted.  We did not have to worry about which particular computer -- referred to as nodes -- these would be deployed to.

`kubectl apply -f api-deployment.yaml`

<img src="./deployment-and-nodes.jpg" width="70%">

> So in the diagram above, Kubernetes distributed the pods across to different nodes (or computers) -- and as developers, we were happy to let Kubernetes decide on where to place this pod. 

In addition, we didn't even need to start the containers up on the pods ourselves.  We simply applied our yaml file and watched Kubernetes perform this for us.  Now creating these pods may seem like this is a minimal task that Kubernetes performs for us, but there is a good amount of work involved.  We can see this with the following command.

`kubectl describe pods`

<img src="./kubectl-describe-pods.png" width="90%">

So we can see that Kubernetes had to move through the steps of pulling our image, creating and then starting our container.  And also deployed the pods to nodes of it's choosing.

### Updating the number of Containers

Things become more interesting when we say want to scale back the number of containers.  To do so, we can just change the number of replicas in our deployment yaml file, apply the change, and then Kubernetes will determine which container to shut down and terminate it.

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: api-deployment
spec:
  replicas: 1
  selector:
    matchLabels:
      component: api
  template:
    metadata:
      labels:
        component: api
    spec:
      containers:
        - name: flask-api
          image: jek2141/flask_api
          ports:
          - containerPort: 5000
```

`kubectl apply -f api-deployment.yaml`

<img src="./terminating-container.png" width="70%">

> Notice that the status now changed to terminating.

So once again, Kubernetes does the job of determining which pod to terminate, finding the pod on whichever machine it exists, and then removing the pod.  After a minute or two, we'll see that the pod has disappeared.

**1. Before applying the update** 

<img src="./deployment-and-nodes.jpg" width="70%">

**2. After applying the update**

<img src="./api-deployment-updated.jpg" width="70%">

> So with Kubernetes we do not need to worry about which nodes our pods are deployed on because Kubernetes will find them and determine which ones to remove.  Without Kubernetes we would have to find the container on the correct node, and then remove it ourselves.  

### Self Healing

One additional benefit that we can see from our deployment is self-healing.  For example, let's say our deployment specified there should be two replicas of a pod.  If one of those replicas or the machine it's running on crashes, Kubernetes will see that only one is running when we want two.  So Kubernetes will get to work creating a new pod to replace it.   

Let's try this out.  We'll have our deployment specify two replicas of our pod, and then apply the deployment.

<img src="./two-pods.png" width="60%">

Next, we'll delete one of the pods.

<img src="./delete-pods.png" width="60%">

Now we did command Kubernetes to delete one of the pods.  But the deployment still specifies there should be two pods.  Kubernetes will notice that there is one fewer pod running then specified in the deployment and create a new pod to rectify this.

> Notice in the image below, that even though one of the pods is now terminating, another pod is up and running to create it's place.

<img src="./kube-delete-status.png" width="60%">

After a few moments, the earlier pod has been fully deleted, and all we have left is our two pods.

<img src="./kube-status-updated.png" width="60%">

So this is pretty special.  Even if one of our machines crashes in the middle of the night, Kubernetes will notice that there is a pod missing, and create a new one on a different machine to replace it.

### Rollouts and Rollbacks

Deployments also keep track of the changes we made in our template.  For example, let's check the number of revisions that were applied in the deployment's history.

<img src="./deployment-rollout-history.png" width="70%">

Ok, so we have two revisions in the deployment --one and two.  From here, let's inspect what occurred in each deployment.  

> `kubectl rollout history deployment --revision=1`

<img src="./revision-1.png" width="70%">

We see that in revision 1 we just specified the image.  And in the revision 2, we added in the port.

<img src="./revision-2.png" width="60%">

Now if, we decide we want to go back to revision one, we can.

<img src="./rollback-deployment.png" width="90%">

> So this would be quite useful, if we applied an update that then caused something to fail, we can simply rollback the changes.

### Summary

In this lesson, we walked through some of the benefits from using deployments, and Kubernetes in general.  As we saw, with deployments, we simply specify the numbers of replicas that we want in our deployment, and Kubernetes will work to achieve this.  For example, if we want to decrease the number of replicas, we change our deployment file and apply the changes.  Kubernetes will do the work of determining which pod to eliminate and find the pod on whichever computer it lives, and destroy the pod.  So this allows to ignore the hardware issues of determining which pod lives on which machine.  We see additional benefits when say a machine a pod is running on accidentally shutsdown -- Kunbernetes will notice there are not the same pods running as specified in the deployment and will create a new pod on a different computer.